In [1]:
type Mondrian_Node
    children::Array
    parent::Array
    τ::Float64
end

function Mondrian_Node()
    N = Mondrian_Node([],[],0)
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node([], [], τ)
    return N
end

type Mondrian_Tree
    nodes::Array
    leaves::Array
end

In [2]:
include("Axis_Aligned_Box.jl");

In [35]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    nodes = []
    leaves = []
    δ = []
    ζ = []
    τ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0)
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ, nodes, leaves)
    return Mondrian_Tree(nodes,leaves)
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ, nodes, leaves)
    push!(nodes,j)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if length(nodes)==1
        τₚ = 0
    else
        τₚ = (j.parent[1]).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        push!(δ,d)
        push!(ζ,x)
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(τₚ+E)
        push!(left.parent,j)
        right = Mondrian_Node(τₚ+E)
        push!(right.parent,j)
        push!(j.children,left)
        push!(j.children,right)
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ, nodes, leaves)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ, nodes, leaves)
    else
        j.τ = λ
        push!(leaves,j)
    end
end

Sample_Mondrian_Block (generic function with 2 methods)

In [36]:
X = [1 2 3; 4 5 6; 7 8 9]
Y = [1,2,3]
D = (X,Y)

([1 2 3; 4 5 6; 7 8 9], [1, 2, 3])

In [43]:
M = Sample_Mondrian_Tree(1,D);